In [179]:
from pyspark.sql.functions import sum, mean, udf, col, round, greatest, count, when
from pyspark.sql.types import DoubleType
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

## Download the new far lookup table

In [148]:
# ! curl -O https://raw.githubusercontent.com/NYCPlanning/db-pluto/master/pluto_build/data/dcp_zoning_maxfar.csv
# ! mv dcp_zoning_maxfar.csv data/dcp_zoning_maxfar.csv

## Loading data
Note here we are casting the far values to double, so that ```"-"``` can get forced into ```Null```

In [180]:
df = spark.read.csv('data/pluto.csv', header=True)
new_far = spark.read.csv('data/dcp_zoning_maxfar.csv', header=True)
df = df.select([col(A).alias(A.lower()) for A in df.columns])

#type conversion, '-' --> null
for A in ['residfar','commfar']:
    new_far = new_far.withColumn(A, col(A).cast(DoubleType()))
    df = df.withColumn(A, col(A).cast(DoubleType()))

#create two copies
new_far1 = new_far.select([col(A).alias(A.lower()+'_1') for A in ['zonedist','residfar','commfar']])
new_far2 = new_far.select([col(A).alias(A.lower()+'_2') for A in ['zonedist','residfar','commfar']])

## Generating new maxfar and pctunbuilt

If after merging on zonedist1, residfar_new or commfar_new are null, then we will use the corresponding zonedist2 residfar and commfar as residfar_new and commfar_new

Below udf checks if A (zonedist1 far value) is null, then we will pick B (zonedist2 far value)

In [181]:
@udf
def pick_value(A,B):
    if not A: 
        return B
    else: 
        return A
    
@udf
def greater(A, B): 
    if not A: 
        if not B: 
            return 0
        else: 
            return B
    if not B: 
        if not A: 
            return 0
        else:
            return A
    if A > B: 
        return A
    else: 
        return B

### Procedures:

1. for each bbl, do a left join using both zonedist1 and zonedist2 as key
2. for each bbl, fill out residfar_new and commfar_new by applying udf pick_value
3. create maxfar = max(residfar, commfar), maxfar_new = max(residfar_new, commfar_new)
4. create pctunbuilt and pctunbuilt_new
5. filter bbls by the following: 
    1. unitsres <= 6
    2. not a landmark
    3. not irregular lot
    4. not house of worship
    5. not public institutions
    6. no easement

In [182]:
df = df.join(new_far1, df['zonedist1'] == new_far1['zonedist_1'], how='left')\
       .join(new_far2, df['zonedist2'] == new_far2['zonedist_2'], how='left')\
       .withColumn('builtfar', col('builtfar').cast(DoubleType()))\
       .withColumn('residfar_new', pick_value(col('residfar_1'), col('residfar_2')))\
       .withColumn('commfar_new', pick_value(col('commfar_1'), col('commfar_2')))\
       .withColumn('maxfar', greatest(col('residfar').cast(DoubleType()), 
                                      col('commfar').cast(DoubleType())))\
       .withColumn('pctunbuilt', (col('maxfar') - col('builtfar')) / col('maxfar'))\
       .withColumn('maxfar_new', greatest(col('residfar_new').cast(DoubleType()), 
                                          col('commfar_new').cast(DoubleType())))\
       .withColumn('pctunbuilt_new', (col('maxfar_new') - col('builtfar')) / col('maxfar_new'))\
       .filter(col('unitsres').cast(DoubleType()) <= 6)\
       .filter(col('landmark').isNull())\
       .filter(col('irrlotcode') != 'Y')\
       .filter(~col ('bldgclass').like('M%'))\
       .filter(col('landuse') != '08')\
       .filter(col('easements').cast(DoubleType()) <= 0)\

In [183]:
# df = df.join(new_far1, df['zonedist1'] == new_far1['zonedist_1'], how='left')\
#        .join(new_far2, df['zonedist2'] == new_far2['zonedist_2'], how='left')\
#        .withColumn('builtfar', col('builtfar').cast(DoubleType()))\
#        .withColumn('residfar_new', pick_value(col('residfar_1'), col('residfar_2')))\
#        .withColumn('commfar_new', pick_value(col('commfar_1'), col('commfar_2')))\
#        .withColumn('maxfar', greater(col('residfar').cast(DoubleType()), 
#                                       col('commfar').cast(DoubleType())))\
#        .withColumn('pctunbuilt', (col('maxfar') - col('builtfar')) / col('maxfar'))\
#        .withColumn('maxfar_new', greater(col('residfar_new').cast(DoubleType()), 
#                                           col('commfar_new').cast(DoubleType())))\
#        .withColumn('pctunbuilt_new', (col('maxfar_new') - col('builtfar')) / col('maxfar_new'))\
#        .filter(col('unitsres').cast(DoubleType()) <= 6)\
#        .filter(col('landmark').isNull())\
#        .filter(col('irrlotcode') != 'Y')\
#        .filter(~col ('bldgclass').like('M%'))\
#        .filter(col('landuse') != '08')\
#        .filter(col('easements').cast(DoubleType()) <= 0)\

In [158]:
df.select('maxfar', 'maxfar_new', 'pctunbuilt', 'pctunbuilt_new', 'builtfar').schema

StructType(List(StructField(maxfar,StringType,true),StructField(maxfar_new,StringType,true),StructField(pctunbuilt,DoubleType,true),StructField(pctunbuilt_new,DoubleType,true),StructField(builtfar,DoubleType,true)))

In [192]:
df.filter((col('residfar') == 0)&
            (col('commfar') == 0) & 
            (col('residfar_new') != 0)&
            (col('commfar_new') != 0))\
  .select('zonedist1').groupBy('zonedist1').agg(count('zonedist1')).show()

+---------+----------------+
|zonedist1|count(zonedist1)|
+---------+----------------+
|  M1-6/R9|              18|
|     PARK|               1|
+---------+----------------+



In [199]:
df.filter(((col('maxfar_new')-col('maxfar'))/col('maxfar') > 0.4))\
  .select('zonedist1').groupBy('zonedist1')\
  .agg(count('zonedist1').alias('counts'))\
  .sort(col('counts').desc()).show()

+---------+------+
|zonedist1|counts|
+---------+------+
|     R3-2|    79|
|     M1-1|    76|
|      R6B|    71|
|      R3A|    66|
|       R5|    59|
|      R3X|    36|
|     C8-1|    21|
|      R5B|    18|
|       R2|    13|
|     C8-3|    11|
|     M1-5|     9|
|     M1-4|     8|
|     R3-1|     8|
|     R7-1|     7|
|     C8-2|     6|
|       R6|     6|
| M1-4/R7D|     5|
|     M1-2|     4|
|       R4|     4|
|      R8B|     4|
+---------+------+
only showing top 20 rows



In [193]:
df.agg(sum(when(col('pctunbuilt').isNull(), 1)).alias('pctunbuilt_null'), 
      sum(when(col('pctunbuilt_new').isNull(), 1)).alias('pctunbuilt_new_null')).show()

+---------------+-------------------+
|pctunbuilt_null|pctunbuilt_new_null|
+---------------+-------------------+
|           3522|               3453|
+---------------+-------------------+



## BBL counts for zonedist with maxfar increase

In [160]:
df.filter(col('maxfar') < col('maxfar_new'))\
    .groupBy('zonedist1')\
    .agg(count('zonedist1').alias('counts'))\
    .sort(col('counts').desc()).show(df.count(), False)

+---------+------+
|zonedist1|counts|
+---------+------+
|R8A      |591   |
|M1-1     |133   |
|R3-2     |79    |
|R6B      |71    |
|R3A      |66    |
|R5       |59    |
|PARK     |52    |
|R4       |51    |
|C8-1     |51    |
|R6       |43    |
|R3X      |36    |
|C8-2     |36    |
|R4A      |20    |
|M1-4/R8A |20    |
|R4B      |19    |
|M1-6/R9  |18    |
|R5B      |18    |
|R2       |13    |
|R7-1     |11    |
|C8-3     |11    |
|M1-2     |10    |
|R6A      |10    |
|R3-1     |8     |
|R4-1     |7     |
|M1-4/R7X |7     |
|M1-4/R7D |5     |
|M1-5     |4     |
|R2A      |4     |
|M1-4     |3     |
|M3-1     |2     |
|R7-2     |2     |
|M1-1/R6A |2     |
|R8B      |2     |
|C8-4     |2     |
|M1-4D    |1     |
|R8       |1     |
|R10      |1     |
|R7A      |1     |
|R5D      |1     |
|M2-1     |1     |
+---------+------+



## Average Far for = residential

In [134]:
df.filter(col('zonedist1').like('R%'))\
   .select('maxfar_new', 'maxfar', 'builtfar')\
   .agg(mean('maxfar_new'),mean('maxfar'),mean('builtfar')).show()

+------------------+------------------+------------------+
|   avg(maxfar_new)|       avg(maxfar)|     avg(builtfar)|
+------------------+------------------+------------------+
|1.1086672562662478|1.1666834392287233|0.8391015326831988|
+------------------+------------------+------------------+



## Softsites changes by borough counts

In [161]:
# By Borough Comparison
df.groupBy('borough')\
    .agg(sum(when(col('pctunbuilt_new') >= 0.5, 1)).alias('softsites_new'),
         sum(when(col('pctunbuilt') >= 0.5, 1)).alias('softsites'))\
    .withColumn('diff', col('softsites_new')-col('softsites'))\
    .sort(col('diff').desc()).show()

+-------+-------------+---------+-----+
|borough|softsites_new|softsites| diff|
+-------+-------------+---------+-----+
|     MN|         6140|     6106|   34|
|     BX|        20242|    21727|-1485|
|     BK|        48361|    50568|-2207|
|     SI|        18810|    23264|-4454|
|     QN|        33537|    41283|-7746|
+-------+-------------+---------+-----+



## Softsites changes by CD counts (top increase)

In [136]:
df.groupBy('cd')\
    .agg(sum(when(col('pctunbuilt_new') >= 0.5, 1)).alias('softsites_new'),
         sum(when(col('pctunbuilt') >= 0.5, 1)).alias('softsites'))\
    .withColumn('diff', col('softsites_new')-col('softsites'))\
    .sort(col('diff').desc()).show()

+---+-------------+---------+----+
| cd|softsites_new|softsites|diff|
+---+-------------+---------+----+
|111|          942|      923|  19|
|302|         1018|     1008|  10|
|301|         2927|     2917|  10|
|308|         1777|     1773|   4|
|206|         1670|     1666|   4|
|108|          976|      972|   4|
|309|         2402|     2399|   3|
|303|         2916|     2913|   3|
|102|          262|      259|   3|
|104|          620|      617|   3|
|201|         1568|     1566|   2|
|106|          538|      536|   2|
|110|          810|      808|   2|
|204|         1300|     1298|   2|
|595|           18|       16|   2|
|304|         2599|     2597|   2|
|202|         1459|     1457|   2|
|316|         3911|     3909|   2|
|205|         1252|     1250|   2|
|484|            9|        7|   2|
+---+-------------+---------+----+
only showing top 20 rows



In [172]:
df.filter(col('cd') == '111')\
  .select('bbl', 'pctunbuilt_new','pctunbuilt','xcoord', 'ycoord')\
  .toPandas().to_csv('cd111_softsites.csv', index = False)

## Softsites changes by borough counts (top decrease)

In [137]:
df.groupBy('cd')\
    .agg(sum(when(col('pctunbuilt_new') >= 0.5, 1)).alias('softsites_new'),
         sum(when(col('pctunbuilt') >= 0.5, 1)).alias('softsites'))\
    .withColumn('diff', col('softsites_new')-col('softsites'))\
    .filter((~col('diff').isNull())).sort(col('diff')).show()

+---+-------------+---------+-----+
| cd|softsites_new|softsites| diff|
+---+-------------+---------+-----+
|412|         4827|     6697|-1870|
|503|         6591|     8251|-1660|
|501|         6623|     8234|-1611|
|502|         5577|     6762|-1185|
|413|         4771|     5918|-1147|
|410|         1881|     2919|-1038|
|318|         2686|     3721|-1035|
|407|         3080|     3906| -826|
|405|         2518|     3169| -651|
|212|         3408|     4003| -595|
|315|         2686|     3205| -519|
|409|         1861|     2309| -448|
|408|         1501|     1925| -424|
|414|         2463|     2876| -413|
|210|         2016|     2428| -412|
|211|         1779|     2180| -401|
|411|         2136|     2524| -388|
|317|         4578|     4842| -264|
|311|         2026|     2166| -140|
|403|         1208|     1342| -134|
+---+-------------+---------+-----+
only showing top 20 rows



In [173]:
df.filter(col('cd') == '412')\
  .select('bbl', 'pctunbuilt_new','pctunbuilt','xcoord', 'ycoord')\
  .toPandas().to_csv('cd412_softsites.csv', index = False)

In [165]:
df.groupBy('cd')\
    .agg(sum(when(col('pctunbuilt_new') >= 0.5, 1)).alias('softsites_new'),
         sum(when(col('pctunbuilt') >= 0.5, 1)).alias('softsites'))\
    .withColumn('diff', col('softsites_new')-col('softsites')).toPandas().to_csv('softsites_cd.csv')

## Changes in percent unbuilt

In [138]:
df.agg(sum(when(col('pctunbuilt_new') > col('pctunbuilt'), 1)).alias('pctunbuilt_increase'), 
      sum(when(col('pctunbuilt_new') == col('pctunbuilt'), 1)).alias('pctunbuilt_unchanged'),
      sum(when(col('pctunbuilt_new') < col('pctunbuilt'), 1)).alias('pctunbuilt_decrease'),).show()

+-------------------+--------------------+-------------------+
|pctunbuilt_increase|pctunbuilt_unchanged|pctunbuilt_decrease|
+-------------------+--------------------+-------------------+
|               1208|              366040|             303847|
+-------------------+--------------------+-------------------+



## Changes in maxfar

In [139]:
df.groupBy().agg(sum(when(col('maxfar_new') > col('maxfar'), 1)).alias('increase'), 
      sum(when(col('maxfar_new') == col('maxfar'), 1)).alias('unchanged'),
      sum(when(col('maxfar_new') < col('maxfar'), 1)).alias('decrease'),).show()

+--------+---------+--------+
|increase|unchanged|decrease|
+--------+---------+--------+
|    1496|   356678|  314489|
+--------+---------+--------+



## Percent unbuilt breakdown

In [140]:
df.agg(sum(when(col('pctunbuilt') > 0.5, 1)).alias('old>0.5'),
       sum(when(col('pctunbuilt') == 0.5, 1)).alias('old=0.5'),
       sum(when(col('pctunbuilt') < 0.5, 1)).alias('old<0.5'),
       sum(when(col('pctunbuilt_new') > 0.5, 1)).alias('new>0.5'),
       sum(when(col('pctunbuilt_new') == 0.5, 1)).alias('new=0.5'),
       sum(when(col('pctunbuilt_new') < 0.5, 1)).alias('new<0.5')).show()

+-------+-------+-------+-------+-------+-------+
|old>0.5|old=0.5|old<0.5|new>0.5|new=0.5|new<0.5|
+-------+-------+-------+-------+-------+-------+
| 136442|   6506| 528148| 121755|   5335| 544075|
+-------+-------+-------+-------+-------+-------+



In [141]:
df.select('zonedist1','maxfar', 'maxfar_new', 'builtfar')\
.filter((col('maxfar_new').cast(DoubleType()) < col('builtfar').cast(DoubleType())) &
        (col('maxfar') > col('builtfar'))).distinct().show()

+---------+------+----------+--------+
|zonedist1|maxfar|maxfar_new|builtfar|
+---------+------+----------+--------+
|     R3-1|   0.6|       0.5|    0.55|
|      R3X|   0.6|       0.5|    0.58|
|      R2X|  1.02|      0.85|    0.95|
|      R3X|   0.6|       0.5|    0.53|
|    M1-1D|  1.65|       1.0|    1.27|
|     R3-1|   0.6|       0.5|    0.54|
|     R3-2|   0.6|       0.5|    0.59|
|       C3|   0.6|       0.5|    0.58|
|      R4A|   0.9|      0.75|    0.86|
|      R4A|   0.9|      0.75|    0.85|
|       R4|   0.9|      0.75|    0.83|
|    M1-1D|  1.65|       1.0|    1.51|
|    M1-1D|  1.65|       1.0|    1.04|
|       R4|   0.9|      0.75|    0.84|
|    M1-1D|  1.65|       1.0|    1.16|
|    M1-1D|  1.65|       1.0|    1.22|
|    M1-1D|  1.65|       1.0|    1.49|
|     R3-2|   0.6|       0.5|    0.56|
|     R4-1|   0.9|      0.75|    0.84|
|      R3A|   0.6|       0.5|    0.54|
+---------+------+----------+--------+
only showing top 20 rows



In [142]:
df.filter((col('pctunbuilt') < 0) &
          (col('pctunbuilt_new') > 0))\
.select('zonedist1','maxfar', 'maxfar_new', 'pctunbuilt', 'pctunbuilt_new', 'builtfar').distinct().show()

+---------+------+----------+--------------------+-------------------+--------+
|zonedist1|maxfar|maxfar_new|          pctunbuilt|     pctunbuilt_new|builtfar|
+---------+------+----------+--------------------+-------------------+--------+
|     M1-1|   1.0|      3.44|-0.33000000000000007| 0.6133720930232558|    1.33|
|      R6B|   2.0|       3.0|-0.02000000000000...|               0.32|    2.04|
|     R3-2|   0.6|       1.0|-0.01666666666666...|               0.39|    0.61|
|     C8-1|   1.0|      2.43| -0.3999999999999999| 0.4238683127572017|     1.4|
|     M1-1|   1.0|      2.43|               -0.78|0.26748971193415644|    1.78|
|     M1-1|   1.0|       2.0| -0.3400000000000001|0.32999999999999996|    1.34|
|     R3-2|   0.6|       2.0|-0.21666666666666667|              0.635|    0.73|
|       R5|  1.25|       3.4|-0.15199999999999997| 0.5764705882352941|    1.44|
|       R2|   0.5|       1.0| -0.3400000000000001|0.32999999999999996|    0.67|
|     R3-2|   0.6|       1.0|-0.05000000

In [143]:
df.filter((col('pctunbuilt') > 0) &
          (col('pctunbuilt_new') < 0))\
.select('zonedist1','maxfar', 'maxfar_new', 'pctunbuilt', 'pctunbuilt_new', 'builtfar').distinct().show()

+---------+------+----------+--------------------+--------------------+--------+
|zonedist1|maxfar|maxfar_new|          pctunbuilt|      pctunbuilt_new|builtfar|
+---------+------+----------+--------------------+--------------------+--------+
|      R4A|   0.9|      0.75| 0.14444444444444446|-0.02666666666666669|    0.77|
|      R2X|  1.02|      0.85| 0.12745098039215685|-0.04705882352941...|    0.89|
|     R4-1|   0.9|      0.75| 0.08888888888888896|-0.09333333333333327|    0.82|
|    M1-1D|  1.65|       1.0|  0.3454545454545454|-0.08000000000000007|    1.08|
|      R3X|   0.6|       0.5| 0.08333333333333323|-0.10000000000000009|    0.55|
|    M1-1D|  1.65|       1.0|  0.2363636363636363|               -0.26|    1.26|
|     R3-2|   0.6|       0.5|0.050000000000000044| -0.1399999999999999|    0.57|
|      C3A|   0.6|       0.5| 0.11666666666666659|-0.06000000000000005|    0.53|
|       C3|   0.6|       0.5| 0.09999999999999991|-0.08000000000000007|    0.54|
|    M1-1D|  1.65|       1.0